In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog
import os
%matplotlib widget

# UI Elements
file_path_label = widgets.Label(value="No file selected")
select_file_button = widgets.Button(description="Select File")
analysis_type = widgets.ToggleButtons(
    options=["Monthly", "Range"],
    description="Analysis:",
    style={"button_width": "100px"},
)
period_dropdown = widgets.Dropdown(description="Period:")
start_period_dropdown = widgets.Dropdown(description="Start Period:", layout=widgets.Layout(display="none"))
end_period_dropdown = widgets.Dropdown(description="End Period:", layout=widgets.Layout(display="none"))
upper_bound_input = widgets.IntText(value=15, description="Max Upper:")
execute_button = widgets.Button(description="Execute", button_style="success")
output = widgets.Output()

# Event for analysis type toggle
def toggle_analysis_type(change):
    if change["new"] == "Monthly":
        start_period_dropdown.layout.display = "none"
        end_period_dropdown.layout.display = "none"
        period_dropdown.layout.display = "block"
    elif change["new"] == "Range":
        start_period_dropdown.layout.display = "block"
        end_period_dropdown.layout.display = "block"
        period_dropdown.layout.display = "none"

analysis_type.observe(toggle_analysis_type, names="value")



# Function to load data from the selected file
def load_data(file_path):
    try:
        data = pd.ExcelFile(file_path)
        sheet1_data = data.parse("Sheet1")
        sheet1_data["Start_Date_time"] = pd.to_datetime(sheet1_data["Start_Date_time"], errors="coerce")
        return sheet1_data
    except Exception as e:
        return str(e)

# Function to get available periods in the dataset
def get_available_periods(data):
    if "Start_Date_time" in data.columns:
        data["YearMonth"] = data["Start_Date_time"].dt.to_period("M").astype(str)
        return sorted(data["YearMonth"].dropna().unique())
    return []

# Function to create frequency table
def create_frequency_table(data, period=None, start_period=None, end_period=None, max_upper=10):
    if period:
        data_filtered = data[data["Start_Date_time"].dt.to_period("M") == period]
    elif start_period and end_period:
        data_filtered = data[(data["Start_Date_time"].dt.to_period("M") >= start_period) &
                             (data["Start_Date_time"].dt.to_period("M") <= end_period)]
    else:
        return None

    # Exclude "Self Practice"
    data_filtered = data_filtered[~data_filtered["Class_Name"].str.contains("Self Practice", case=False, na=False)]

    # Calculate booking frequencies
    booking_frequencies = data_filtered.groupby("Id_Person").size()

    # Create frequency table
    max_upper = int(max_upper)
    table = pd.DataFrame({
        "Freq": list(range(1, max_upper + 1)) + [f">{max_upper}"],
        "#Students": [sum(booking_frequencies == i) for i in range(1, max_upper + 1)] + [sum(booking_frequencies > max_upper)],
        "Cum 1->": [sum(booking_frequencies <= i) for i in range(1, max_upper + 1)] + [len(booking_frequencies)],
        "Cum ->End": [len(booking_frequencies) - sum(booking_frequencies <= i) for i in range(1, max_upper + 1)] + [sum(booking_frequencies > max_upper)],
    })

    # Add student IDs or names based on frequency bands
    def get_student_details(freq):
        if isinstance(freq, str) and freq.startswith(">"):
            ids = booking_frequencies[booking_frequencies > max_upper].index
            names = data_filtered[data_filtered["Id_Person"].isin(ids)]["FirstName"].drop_duplicates().tolist()
            return ", ".join([f"{name} : {id}" for name, id in zip(names, ids)])
        elif isinstance(freq, int) and freq <= 3:
            ids = booking_frequencies[booking_frequencies == freq].index
            names = data_filtered[data_filtered["Id_Person"].isin(ids)]["FirstName"].drop_duplicates().tolist()
            return ", ".join(map(str, ids)), ", ".join([f"{name} : {id}" for name, id in zip(names, ids)])
        else:
            ids = booking_frequencies[booking_frequencies == freq].index
            names = data_filtered[data_filtered["Id_Person"].isin(ids)]["FirstName"].drop_duplicates().tolist()
            return ", ".join([f"{name} : {id}" for name, id in zip(names, ids)])

    table["Details"] = [get_student_details(freq) for freq in table["Freq"]]

    return table

def plot_histogram_with_tooltip(table):
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(table["Freq"].astype(str), table["#Students"], color='skyblue', edgecolor='black')

    # Calculate mean and median based on the number of bookings per student
    student_counts = table.loc[table["Freq"].apply(lambda x: str(x).isdigit()), ["Freq", "#Students"]].copy()
    student_counts["Freq"] = student_counts["Freq"].astype(int)
    expanded = student_counts.loc[student_counts.index.repeat(student_counts["#Students"])]
    mean_val = expanded["Freq"].mean()
    median_val = expanded["Freq"].median()

    # Add mean and median lines
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=1, label=f'Mean: {mean_val:.2f}')
    ax.axvline(median_val, color='green', linestyle='--', linewidth=1, label=f'Median: {median_val:.2f}')

    # Add numbers on top of bars
    for bar, student_count in zip(bars, table["#Students"]):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height + 0.5, str(student_count), ha='center', fontsize=10)

    # Add tooltips using mplcursors
    import mplcursors
    cursor = mplcursors.cursor(bars, hover=True)

    @cursor.connect("add")
    def on_add(sel):
        freq = table.iloc[sel.index]["Freq"]
        details = table.iloc[sel.index]["Details"]
        sel.annotation.set(
            text=f"Freq: {freq}\nDetails: {details}",
            fontsize=10,
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8)
        )

    ax.set_xlabel("Frequency of Bookings", fontsize=12)
    ax.set_ylabel("Number of Students", fontsize=12)
    ax.set_xticks(range(len(table)))
    ax.set_xticklabels(table["Freq"].astype(str), fontsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    plt.legend()
    plt.tight_layout()
    plt.show()

# Function to select a file
def select_file():
    root = Tk()
    root.withdraw()  # Hide the main tkinter window
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    root.destroy()  # Close the tkinter root window
    return file_path

# Event for file selection
def on_select_file_clicked(b):
    global file_path  # Use the global variable
    file_path = select_file()
    if file_path:
        file_path_label.value = file_path
        try:
            data = pd.ExcelFile(file_path).parse("Sheet1")
            data["Start_Date_time"] = pd.to_datetime(data["Start_Date_time"], errors="coerce")
            periods = sorted(data["Start_Date_time"].dt.to_period("M").astype(str).dropna().unique())
            period_dropdown.options = periods
            start_period_dropdown.options = periods
            end_period_dropdown.options = periods
        except Exception as e:
            file_path_label.value = f"Error loading file: {e}"

# Event for analysis type toggle
def toggle_analysis_type(change):
    if change["new"] == "Monthly":
        start_period_dropdown.layout.display = "none"
        end_period_dropdown.layout.display = "none"
        period_dropdown.layout.display = "block"
    elif change["new"] == "Range":
        start_period_dropdown.layout.display = "block"
        end_period_dropdown.layout.display = "block"
        period_dropdown.layout.display = "none"

analysis_type.observe(toggle_analysis_type, names="value")

# Event to execute analysis
def on_execute_button_clicked(b):
    with output:
        output.clear_output()
        file_path = file_path_label.value
        if file_path == "No file selected" or "Error" in file_path:
            print("Please select a valid file.")
            return

        data = pd.ExcelFile(file_path).parse("Sheet1")
        data["Start_Date_time"] = pd.to_datetime(data["Start_Date_time"], errors="coerce")
        max_upper = upper_bound_input.value

        if analysis_type.value == "Monthly":
            period = period_dropdown.value
            table = create_frequency_table(data, period=period, max_upper=max_upper)
            report_title = f"<h3 style='text-align: center;'>Booking Frequency Report for {period}</h3>"
        elif analysis_type.value == "Range":
            start_period = start_period_dropdown.value
            end_period = end_period_dropdown.value
            table = create_frequency_table(data, start_period=start_period, end_period=end_period, max_upper=max_upper)
            report_title = f"<h3 style='text-align: center;'>Booking Frequency Report from {start_period} to {end_period}</h3>"
        else:
            table = None

        if table is not None:
            display(HTML(report_title))



            # Embed inline styles for column-specific widths
            styled_rows = ""
            for _, row in table.iterrows():
                freq = row['Freq']
                ids, tooltip = row['Details'] if isinstance(freq, int) and freq <= 3 else (row['Details'], row['Details'])

                styled_rows += f"<tr title='{tooltip}'>"
                styled_rows += f"<td style='width: 5%; text-align: center; font-size: 14px;'>{row['Freq']}</td>"
                styled_rows += f"<td style='width: 10%; text-align: center; font-size: 14px;'>{row['#Students']}</td>"
                styled_rows += f"<td style='width: 10%; text-align: center; font-size: 14px;'>{row['Cum 1->']}</td>"
                styled_rows += f"<td style='width: 10%; text-align: center; font-size: 14px;'>{row['Cum ->End']}</td>"
                styled_rows += f"<td style='width: 65%; text-align: center; font-size: 14px;'>{ids}</td>"
                styled_rows += "</tr>"

            styled_table = f"""
            <style>
                table {{
                    width: 100%;
                    border-collapse: collapse;
                }}
                th, td {{
                    border: 1px solid black;
                    padding: 8px;
                }}
                th {{
                    text-align: center;
                    font-size: 16px;
                }}
                td {{
                    vertical-align: top;
                }}
            </style>
            <table>
                <thead>
                    <tr>
                        <th style='width: 5%;'>Freq</th>
                        <th style='width: 10%;'>#Students</th>
                        <th style='width: 10%;'>Cum 1-></th>
                        <th style='width: 10%;'>Cum ->End</th>
                        <th style='width: 65%;'>Details</th>
                    </tr>
                </thead>
                <tbody>
                    {styled_rows}
                </tbody>
            </table>
            """
            # Plot histogram
            plot_histogram_with_tooltip(table)
            
            display(HTML(styled_table))

            
# Link buttons to functions
select_file_button.on_click(on_select_file_clicked)
execute_button.on_click(on_execute_button_clicked)

# Layout
ui = widgets.VBox([
    widgets.HBox([select_file_button, file_path_label]),
    widgets.HBox([analysis_type]),
    widgets.HBox([period_dropdown, start_period_dropdown, end_period_dropdown]),
    widgets.HBox([upper_bound_input]),
    execute_button,
    output
])

display(ui)
